<a href="https://colab.research.google.com/github/rjanow/Masterarbeit/blob/main/SOLYS_to_CSV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SOLYS_to_CSV

Hier werden die Sun-Tracker Daten in eine CSV umgewandelt:

In [1]:
# Verbinden mit der Google-Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%matplotlib inline

In [3]:
# Import der benötigten Module
import os
import pandas as pd
import matplotlib.pyplot  as plt

from google.colab import drive

In [4]:
# Pfad zum Ordner der Drive in dem die Messdaten gespeichert sind
folder_path = f'/content/drive/My Drive/Colab_Notebooks/SOLYS_Messdaten'

In [5]:
# Funktion zum Laden aller Dateien in einem Google Drive-Ordner
def load_data_from_drive_folder(folder_path, header_lines=7, encoding = 'cp1252'):

    # Liste, um alle DataFrames zu speichern
    dfs = []

    # Durchlaufen alle Dateien im angegebenen Ordner
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)

        # Überprüfe, ob es eine Datei ist und lade sie
        if os.path.isfile(file_path):
            df = pd.read_csv(file_path, skiprows=header_lines, sep = '\t', encoding=encoding)
            dfs.append(df)

    # Kombinieren aller DataFrames
    combined_df = pd.concat(dfs, ignore_index=True)

    return combined_df

In [6]:
# Aufruf der Funktion zum Einlesen der Messdaten
df = load_data_from_drive_folder(folder_path)

## Löschen und umbenennen der Spalten:

In [9]:
# Löschen der nicht benötigten Splaten
df_Solys = df.drop(columns = ['Unit', 'Unnamed: 10', ' °C', 'V', ' °C.1', 'Unnamed: 9', '171.79166666666666,172.4,185.86666666666667,2.058333333333333,10.658333333333333'], axis=1)

In [12]:
# Umbenennen der Splatennamen
df_Solys.columns = ['Datetime', 'Glo', 'Dif', 'Glo_SPLite', 'Dir', 'Temp']

AttributeError: ignored

In [13]:
# Löschen aller NaN Zeilen
df_Solys = df_Solys.dropna()

In [18]:
print(df_Solys.info(), '\n --------------------- \n', df_Solys.isnull().sum())

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 4324959 entries, 2022-06-21 13:52:30 to 2023-10-26 10:02:50
Data columns (total 5 columns):
 #   Column      Dtype 
---  ------      ----- 
 0   Glo         object
 1   Dif         object
 2   Glo_SPLite  object
 3   Dir         object
 4   Temp        object
dtypes: object(5)
memory usage: 198.0+ MB
None 
 --------------------- 
 Glo           0
Dif           0
Glo_SPLite    0
Dir           0
Temp          0
dtype: int64


In [14]:
# Datetime-Spalte als Index setzen
df_Solys.set_index('Datetime', inplace = True)

In [15]:
# Index in Datetime umwandeln
df_Solys.index = pd.to_datetime(df_Solys.index, format='%d.%m.%Y %H:%M:%S')

In [16]:
df_Solys

,Glo,Dif,Glo_SPLite,Dir,Temp
Datetime,,,,,
2022-06-21 13:52:30,"519,9","402,3","500,1","135,3","26,1"
2022-06-21 13:52:40,"425,5","396,0","425,7","36,1","26,1"
2022-06-21 13:52:50,"397,0","390,8","407,3","12,9","26,3"
2022-06-21 13:53:00,"386,7","386,0","397,7","7,4","26,3"
2022-06-21 13:53:10,"379,3","380,7","391,1","5,1","26,3"
...,...,...,...,...,...
2023-10-26 10:02:10,"133,1","138,7","135,2","2,4","11,8"
2023-10-26 10:02:20,"132,7","138,2","134,8","2,7","11,9"
2023-10-26 10:02:30,"132,3","137,8","134,6","2,9","11,8"


## Messungen in die richtige Reihenfolge bringen:

In [19]:
# Sotieren des Index in chronologischer Reihenfolge
df_Solys.sort_index(ascending=True, inplace = True)

<ipython-input-19-6eb4b87478c7>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_Solys.sort_index(ascending=True, inplace = True)


In [ ]:
df_Solys

In [21]:
# Prüfen, ob die Messdaten in chronologischer Reihenfolge sind
chronologisch = df_Solys.index.is_monotonic_increasing
print(f"Der Index ist in chronologischer Reihenfolge: {chronologisch}")

Der Index ist in chronologischer Reihenfolge: True


In [22]:
for column in df_Solys[['Glo','Dif','Glo_SPLite','Dir','Temp']]:
    try:
        df_Solys[column] = df_Solys[column].str.replace(',', '.').astype(float)
    except ValueError:
        print(f"Spalte {column} konnte nicht in float umgewandelt werden.")

<ipython-input-22-87f8aa3123f0>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_Solys[column] = df_Solys[column].str.replace(',', '.').astype(float)


In [30]:
df_Solys['date'] = df_Solys.index.date
df_Solys['time'] = df_Solys.index.time

<ipython-input-30-e5d881dc4aef>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_Solys['time'] = df_Solys.index.time


## Prüfen, ob die Daten zusammen hängen:

In [24]:
round(df_Solys.describe(), 2).T

,count,mean,std,min,25%,50%,75%,max
Glo,4324959.0,153.22,244.17,0.0,0.0,8.5,217.9,1402.3
Dif,4324959.0,71.32,104.99,0.0,0.0,9.6,110.9,1045.0
Glo_SPLite,4324959.0,155.96,244.26,0.0,0.0,11.9,223.9,1344.7
Dir,4324959.0,150.14,282.80,0.0,1.4,2.0,88.5,1018.7
Temp,4324959.0,15.30,8.08,-12.6,9.8,15.3,20.6,251.9


In [29]:
df_Solys

,Glo,Dif,Glo_SPLite,Dir,Temp,date
Datetime,,,,,,
2022-04-25 08:07:30,243.9,244.9,257.1,2.4,10.7,2022-04-25
2022-04-25 08:07:40,245.8,246.9,259.3,2.4,10.8,2022-04-25
2022-04-25 08:07:50,247.8,248.8,261.3,2.4,10.8,2022-04-25
2022-04-25 08:08:00,249.1,250.0,262.4,2.4,10.8,2022-04-25
2022-04-25 08:08:10,249.5,250.3,262.7,2.4,10.9,2022-04-25
...,...,...,...,...,...,...
2023-11-09 08:18:50,14.1,14.7,15.2,1.5,8.8,2023-11-09
2023-11-09 08:19:00,14.3,14.9,15.4,1.5,8.8,2023-11-09
2023-11-09 08:19:10,14.5,15.0,15.6,1.5,8.8,2023-11-09


In [33]:
# def pruefe_zeitlichen_abstand(df, min_abstand):

#   ergebnisse = []

#   for datum, gruppe in df.groupby(df.index.date):
#         gruppe = gruppe.sort_index()
#         abstaende = gruppe.index.to_series().diff().dropna()
#         erfuellt = all(abstaende >= min_abstand)
#         if erfuellt == False:
#           ergebnisse.append({'Datum': datum, 'Erfuellt': erfuellt})

#   ergebnis_df = pd.DataFrame(ergebnisse)
#   ergebnis_df['Datum'] = pd.to_datetime(ergebnis_df['Datum'])

#   return ergebnis_df

In [36]:
def pruefe_zeitlichen_abstand(df, min_abstand):
    ergebnisse = []

    for datum, gruppe in df.groupby(df.index.date):
        gruppe = gruppe.sort_index()
        abstaende = gruppe.index.to_series().diff().dropna()

        # Finden von Zeitstempeln, bei denen der Abstand zu klein ist
        zu_kleine_abstaende = abstaende[abstaende < min_abstand].index

        # Für jeden zu kleinen Abstand, speichern Sie das Datum, die Uhrzeit und ob der Abstand erfüllt ist
        for zeit in zu_kleine_abstaende:
            ergebnisse.append({'Datum': datum, 'Uhrzeit': zeit.time(), 'Erfuellt': False})

        # Falls es keine zu kleinen Abstände gibt, fügen Sie den Tag mit einem erfüllten Status hinzu
        if zu_kleine_abstaende.empty:
            ergebnisse.append({'Datum': datum, 'Uhrzeit': None, 'Erfuellt': True})

    ergebnis_df = pd.DataFrame(ergebnisse)
    ergebnis_df['Datum'] = pd.to_datetime(ergebnis_df['Datum'])

    return ergebnis_df

In [37]:
# Aufruf der Funktion um zu prüfen, ob die Daten zusammenhängend sind,
ergebnis_df = pruefe_zeitlichen_abstand(df_Solys, pd.Timedelta('10S'))
print(len(pd.unique(df_Solys.index.date)), len(ergebnis_df[ergebnis_df['Erfuellt']]))

504 499


In [40]:
# Tage an denen die Messdaten nicht zusammenhängen finden
print(ergebnis_df[~ergebnis_df['Erfuellt']])

           Datum   Uhrzeit  Erfuellt
96    2022-08-08  06:42:50     False
97    2022-08-08  06:43:00     False
98    2022-08-08  06:43:10     False
99    2022-08-08  06:43:20     False
100   2022-08-08  06:43:30     False
...          ...       ...       ...
36898 2022-08-12  12:56:30     False
36899 2022-08-12  12:56:40     False
36900 2022-08-12  12:56:50     False
36901 2022-08-12  12:57:00     False
36902 2022-08-12  12:57:10     False

[36807 rows x 3 columns]


Tage mit fehlenden Daten:
    
         Datum    	    Erfuellt
1.   96  2022-08-08     False
2.   97  2022-08-09     False
3.   98  2022-08-10     False
4.   99  2022-08-11     False
5.   100 2022-08-12     False

## Plotten der fehlenden Messdaten:

In [47]:
df_plot = df_Solys

In [49]:
def plot_selected_days(df, date_column, value_columns, selected_dates):
    # Stelle sicher, dass das Datum in den DataFrames korrekt formatiert ist
    df_func = df

    df_func[date_column] = pd.to_datetime(df_func[date_column])
    selected_dates = pd.to_datetime(selected_dates)

    # print(selected_dates.dtype, df_func[date_column].dtype)

    # Gruppiere den DataFrame nach dem Datum
    grouped = df_func.groupby(df_func.index.date)

    # Erstelle für jeden ausgewählten Tag ein Plot
    for date, group in grouped:
        for selected_dates in date:
            print(date)

In [50]:
# plot_selected_days(df_plot, 'date', ['Glo', 'Dir'], ['2022-08-11', '2022-08-12'])
# ergebnis_df['Datum'][~ergebnis_df['Erfuellt']]

<ipython-input-49-10bbf2cf93ac>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_func[date_column] = pd.to_datetime(df_func[date_column])


TypeError: ignored

## Daten reduzieren:

Daten reduzieren damit die Auflösung mit der des Spektrometers überein stimmt:

In [51]:
# Durchschnittswerte für 2 Min. berechnen
df_Solys_mean = df_Solys.resample('2T').mean()

<ipython-input-51-757fa21c22f9>:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_Solys_mean = df_Solys.resample('2T').mean()


In [52]:
df_Solys_mean

,Glo,Dif,Glo_SPLite,Dir,Temp
Datetime,,,,,
2022-04-25 08:06:00,245.833333,246.866667,259.233333,2.400000,10.766667
2022-04-25 08:08:00,237.058333,237.666667,249.200000,2.350000,10.658333
2022-04-25 08:10:00,206.466667,206.675000,217.633333,2.316667,10.058333
2022-04-25 08:12:00,188.858333,189.400000,200.900000,2.316667,10.608333
2022-04-25 08:14:00,186.991667,187.616667,199.800000,2.208333,10.391667
...,...,...,...,...,...
2023-11-09 08:10:00,9.975000,10.500000,11.266667,1.400000,8.683333
2023-11-09 08:12:00,10.233333,10.741667,11.291667,1.400000,8.708333
2023-11-09 08:14:00,10.775000,11.275000,11.691667,1.416667,8.716667


## Abspeichern der Messdaten als CSV in der Google-Drive

In [53]:
def save_dataframe_to_drive(dataframe, folder_path, filename):

    # Erstelle den vollen Pfad zur Datei
    full_path = os.path.join(folder_path, filename)

    # Speichere den DataFrame als CSV-Datei auf Google Drive
    dataframe.to_csv(full_path, index=False)

In [54]:
df_file_name = 'Solys_CSV'
save_dataframe_to_drive(df_Solys_mean, folder_path, df_file_name)